In [1]:
%matplotlib inline

In [2]:
from datetime import datetime

In [3]:
#renderer
from PIL import ImageFont
import numpy as np
import cv2
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_

# char render
def render(text, font=None):
    if font is None:
        font = ImageFont.truetype("/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc", 32)
    mask = font.getmask(text)
    size = mask.size[::-1]
    a = np.asarray(mask).reshape(size) / 255
    res = cv2.resize(a, dsize=(32, 32), interpolation=cv2.INTER_CUBIC)
    return res

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/data_utils.py
import torch
import re

class Dictionary(object):
    def __init__(self, max_size=None):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 1
        self.word2idx['⸘'] = 0 # as unk
        self.idx2word[0] = '⸘'
        self.max_size = max_size + 1
    
    def add_word(self, word):
        if not word in self.word2idx and self.idx < self.max_size:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    def __init__(self, max_size=None):
        self.dictionary = Dictionary(max_size=max_size)

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
#         with open(path, 'r') as f:
#             tokens = 0
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 tokens += len(words)
#                 for word in words: 
#                     self.dictionary.add_word(word)  

        # split words to char and add to dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                tokens += len(chars)
                for char in chars:
                    self.dictionary.add_word(char)
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                for char in chars:
                    if char in self.dictionary.word2idx:
                        ids[token] = self.dictionary.word2idx[char]
                        token += 1
                    else:
                        ids[token] = self.dictionary.word2idx['⸘']
                        token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, char_id, char_cnn_o, h):
        # Embed word ids to vectors
        x = self.embed(char_id) + char_cnn_o
        
        # Forward propagate LSTM
        out, h = self.gru(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, h

In [8]:
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    if type(stride) is not tuple:
        stride = (stride, stride)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride[0]) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride[1]) + 1)
    return h, w

# Dai et al. 's CNN glyph encoder
class Dai_CNN(nn.Module):
    def __init__(self, embed_size, input_size=(32, 32)):
        super(Dai_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, (7, 7), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)
        h, w = conv_output_shape(input_size, (7, 7), (2, 2))
        
        self.conv2 = nn.Conv2d(32, 16, (5, 5), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)
        h, w = conv_output_shape((h, w), (5, 5), (2, 2))
                
        self.fc = nn.Linear(16*h*w, embed_size)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.zeros_(self.fc.bias)
        
        self.h, self.w = h, w
        
    def forward(self, char_img):
        b = char_img.size(0)
        x = self.conv1(char_img)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = x.view(-1, 16*self.h*self.w)
        return self.fc(x)

In [9]:
# Hyper-parameters
embed_size = 300
hidden_size = 128
num_layers = 1
num_epochs = 50
batch_size = 16
seq_length = 1024
learning_rate = 1e-3

# Load dataset
corpus = Corpus(max_size=4000)
ids = corpus.get_data('icwb2-data/training/msr_training.utf8', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [10]:
# save char images for reference
for char, idx in corpus.dictionary.word2idx.items():
    np.save(f'char_img/noto_CJK/msr2/{idx}.npy', render(char))

In [11]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
cnn_encoder = Dai_CNN(embed_size).to(device)
model.train()
cnn_encoder.train()
params = list(model.parameters())+list(cnn_encoder.parameters())

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Truncated backpropagation
def detach(state):
    return state.detach()

In [12]:
# Train the charID RNNLM only
for param in cnn_encoder.parameters():
    param.requires_grad = False

for epoch in range(num_epochs//3):
    # Set initial hidden and cell states
    state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Get images
        images = np.zeros((inputs.size(0), inputs.size(1), 32, 32, 1))
        for b, seq in enumerate(ids):
            for s, idx in enumerate(ids[b][i:i+seq_length]):
                images[b, s] = np.load(f'char_img/noto_CJK/msr2/{idx}.npy').reshape(32,32,1)
        images = torch.from_numpy(images).float().to(device) # B N H W C
        images = images.view(-1, 32, 32, 1) # B*N H W C
        images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
        
        # Get encoded images
        cnn_o = cnn_encoder(images)
        cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))
               
        # Forward pass
        state = detach(state)
        outputs, state = model(inputs, cnn_o, state)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        cnn_encoder.zero_grad()
        loss.backward()
        clip_grad_norm_(params, 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 1000 == 0:
            print ('{} Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(datetime.now(), epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

2019-11-20 18:43:37.557970 Epoch [1/50], Step[0/252], Loss: 8.3145, Perplexity: 4082.55
2019-11-20 19:02:37.157942 Epoch [2/50], Step[0/252], Loss: 5.0933, Perplexity: 162.92
2019-11-20 19:21:31.249640 Epoch [3/50], Step[0/252], Loss: 4.6588, Perplexity: 105.51
2019-11-20 19:40:25.892552 Epoch [4/50], Step[0/252], Loss: 4.4404, Perplexity: 84.81
2019-11-20 19:59:18.805417 Epoch [5/50], Step[0/252], Loss: 4.3084, Perplexity: 74.32
2019-11-20 20:18:14.016029 Epoch [6/50], Step[0/252], Loss: 4.2119, Perplexity: 67.49
2019-11-20 20:37:07.423957 Epoch [7/50], Step[0/252], Loss: 4.1382, Perplexity: 62.69
2019-11-20 20:56:01.359370 Epoch [8/50], Step[0/252], Loss: 4.0822, Perplexity: 59.28
2019-11-20 21:14:54.740608 Epoch [9/50], Step[0/252], Loss: 4.0391, Perplexity: 56.77
2019-11-20 21:33:48.683805 Epoch [10/50], Step[0/252], Loss: 4.0040, Perplexity: 54.82
2019-11-20 21:52:42.200207 Epoch [11/50], Step[0/252], Loss: 3.9740, Perplexity: 53.20
2019-11-20 22:11:35.822118 Epoch [12/50], Step[0

In [13]:
# Train the CNN + RNNLM without Char Embedding
for param in model.embed.parameters():
    param.requires_grad = False
for param in cnn_encoder.parameters():
    param.requires_grad = True

for epoch in range(num_epochs//3):
    # Set initial hidden and cell states
    state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Get images
        images = np.zeros((inputs.size(0), inputs.size(1), 32, 32, 1))
        for b, seq in enumerate(ids):
            for s, idx in enumerate(ids[b][i:i+seq_length]):
                images[b, s] = np.load(f'char_img/noto_CJK/msr2/{idx}.npy').reshape(32,32,1)
        images = torch.from_numpy(images).float().to(device) # B N H W C
        images = images.view(-1, 32, 32, 1) # B*N H W C
        images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
        
        # Get encoded images
        cnn_o = cnn_encoder(images)
        cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))
               
        # Forward pass
        state = detach(state)
        outputs, state = model(inputs, cnn_o, state)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        cnn_encoder.zero_grad()
        loss.backward()
        clip_grad_norm_(params, 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 1000 == 0:
            print ('{} Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(datetime.now(), epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

2019-11-20 23:45:58.803667 Epoch [1/50], Step[0/252], Loss: 3.8608, Perplexity: 47.50
2019-11-21 00:05:16.787721 Epoch [2/50], Step[0/252], Loss: 3.8509, Perplexity: 47.03
2019-11-21 00:24:32.874392 Epoch [3/50], Step[0/252], Loss: 3.8423, Perplexity: 46.63
2019-11-21 00:43:44.691961 Epoch [4/50], Step[0/252], Loss: 3.8344, Perplexity: 46.27
2019-11-21 01:02:54.217949 Epoch [5/50], Step[0/252], Loss: 3.8276, Perplexity: 45.95
2019-11-21 01:22:02.032973 Epoch [6/50], Step[0/252], Loss: 3.8214, Perplexity: 45.67
2019-11-21 01:41:10.795258 Epoch [7/50], Step[0/252], Loss: 3.8151, Perplexity: 45.38
2019-11-21 02:00:19.168477 Epoch [8/50], Step[0/252], Loss: 3.8097, Perplexity: 45.14
2019-11-21 02:19:27.276479 Epoch [9/50], Step[0/252], Loss: 3.8047, Perplexity: 44.91
2019-11-21 02:38:36.230828 Epoch [10/50], Step[0/252], Loss: 3.8000, Perplexity: 44.70
2019-11-21 02:57:46.342245 Epoch [11/50], Step[0/252], Loss: 3.7960, Perplexity: 44.52
2019-11-21 03:16:56.739133 Epoch [12/50], Step[0/252

In [14]:
# Train the CNN + RNNLM
for param in model.embed.parameters():
    param.requires_grad = True

for epoch in range(num_epochs//3 + num_epochs%3):
    # Set initial hidden and cell states
    state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Get images
        images = np.zeros((inputs.size(0), inputs.size(1), 32, 32, 1))
        for b, seq in enumerate(ids):
            for s, idx in enumerate(ids[b][i:i+seq_length]):
                images[b, s] = np.load(f'char_img/noto_CJK/msr2/{idx}.npy').reshape(32,32,1)
        images = torch.from_numpy(images).float().to(device) # B N H W C
        images = images.view(-1, 32, 32, 1) # B*N H W C
        images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
        
        # Get encoded images
        cnn_o = cnn_encoder(images)
        cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))
               
        # Forward pass
        state = detach(state)
        outputs, state = model(inputs, cnn_o, state)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        cnn_encoder.zero_grad()
        loss.backward()
        clip_grad_norm_(params, 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 1000 == 0:
            print ('{} Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(datetime.now(), epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

2019-11-21 04:52:43.056579 Epoch [1/50], Step[0/252], Loss: 3.7767, Perplexity: 43.67
2019-11-21 05:11:59.234767 Epoch [2/50], Step[0/252], Loss: 3.7580, Perplexity: 42.86
2019-11-21 05:31:16.894344 Epoch [3/50], Step[0/252], Loss: 3.7496, Perplexity: 42.50
2019-11-21 05:50:36.277841 Epoch [4/50], Step[0/252], Loss: 3.7431, Perplexity: 42.23
2019-11-21 06:09:53.984507 Epoch [5/50], Step[0/252], Loss: 3.7371, Perplexity: 41.98
2019-11-21 06:29:14.124631 Epoch [6/50], Step[0/252], Loss: 3.7318, Perplexity: 41.75
2019-11-21 06:48:33.716801 Epoch [7/50], Step[0/252], Loss: 3.7266, Perplexity: 41.54
2019-11-21 07:07:51.728401 Epoch [8/50], Step[0/252], Loss: 3.7218, Perplexity: 41.34
2019-11-21 07:27:10.989150 Epoch [9/50], Step[0/252], Loss: 3.7174, Perplexity: 41.16
2019-11-21 07:46:28.866946 Epoch [10/50], Step[0/252], Loss: 3.7133, Perplexity: 40.99
2019-11-21 08:05:46.976262 Epoch [11/50], Step[0/252], Loss: 3.7093, Perplexity: 40.82
2019-11-21 08:25:05.613537 Epoch [12/50], Step[0/252

In [15]:
model.eval()
cnn_encoder.eval()

perplexity = .0
num_step = 0
for i in range(0, ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = ids[:, i:i+seq_length].to(device)
    targets = ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), 32, 32, 1))
    for b, seq in enumerate(ids):
        for s, idx in enumerate(ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr2/{idx}.npy').reshape(32,32,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, 32, 32, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Get encoded images
    cnn_o = cnn_encoder(images)
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(inputs, cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))

    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Train Perplexity: {perplexity / num_step}")

Train Perplexity: 43.23622297305548


In [16]:
test_ids = corpus.get_data('icwb2-data/testing/msr_test.utf8', batch_size)
# filter out unknown character
test_ids = test_ids.view(-1)
mask = test_ids < vocab_size
test_ids = test_ids[mask]
num_batches = test_ids.size(0) // batch_size
test_ids = test_ids[:num_batches*batch_size]
test_ids = test_ids.view(batch_size, -1)

In [17]:
perplexity = .0
num_step = 0
for i in range(0, test_ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = test_ids[:, i:i+seq_length].to(device)
    targets = test_ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), 32, 32, 1))
    for b, seq in enumerate(test_ids):
        for s, idx in enumerate(test_ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr2/{idx}.npy').reshape(32,32,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, 32, 32, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Get encoded images
    cnn_o = cnn_encoder(images)
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(inputs, cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))
    
    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Test Perplexity: {perplexity / num_step}")

Test Perplexity: 54.09053687480516
